## 

# Crew AI - Book Search

## Import libraries

In [1]:
import os
from crewai import Crew, Task, Agent

# Warning control
import warnings
warnings.filterwarnings('ignore')

## Configuration

In [2]:
# ------ Ollama - Local---------#
os.environ["LOCAL_EMBEDDINGS"]="http://localhost:11434/api/embeddings"
embeddings_model = "nomic-embed-text:v1.5"
os.environ["LOCAL_EMBEDDINGS_MODEL"]=embeddings_model # documentar como definir variáveis de ambiente

# ------ OpenAI - API ---------#
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# ------ DEBUG ---------#
os.environ["DEBUG"]="False"
debug = os.getenv("DEBUG_MODE", "False")
if debug.lower() == "true":
    VERBOSE = 2
else:
    VERBOSE = False

### Crew - Busca no contexto

In [3]:
from langchain_openai import ChatOpenAI
from search.context import utils

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

from crewai_tools import tool

# Configurações locais
LOCAL_EMBEDDINGS_URL = "http://localhost:11434/api/embeddings"
EMBEDDINGS_MODEL = "nomic-embed-text:v1.5"
os.environ["LOCAL_EMBEDDINGS"] = LOCAL_EMBEDDINGS_URL
os.environ["LOCAL_EMBEDDINGS_MODEL"] = EMBEDDINGS_MODEL

# Configurações da API OpenAI
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Configurações de DEBUG
os.environ["DEBUG"] = "False"
DEBUG_MODE = os.getenv("DEBUG_MODE", "False").lower() == "true"
VERBOSE = 2 if DEBUG_MODE else False

# Constantes Globais
AVATARS = {"commentary": "./assets/e-book.png", "bible": "./assets/biblia.png",
           "revisor": "./assets/revisor.png"}
VECTOR_STORE = 'chroma'
PATH_BOOKS = f"./vector-store/{VECTOR_STORE}/books"
PATH_BIBLE = f"./vector-store/{VECTOR_STORE}/bible"
EMBEDDINGS_MODEL = os.getenv("LOCAL_EMBEDDINGS_MODEL")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
DEBUG = os.getenv("DEBUG_MODE", "False").lower() == "true"
VERBOSE = 2 if DEBUG else False


# ------- TOOLS ------- #

@tool("Pesquisar em Livros")
def library_tool(text: str) -> str:
    """Retorna livros que possuem conteúdo relacionado com termo ou assunto buscado."""
    result = ""
    db = utils.retriever_context(embeddings_model=EMBEDDINGS_MODEL, path_books=PATH_BOOKS, vector_store=VECTOR_STORE)
    documents = db.similarity_search(text)
    for row in documents:
        page_content = row.page_content
        source = row.metadata['source'].replace('./books/', '').replace('.pdf', '')
        page = row.metadata['page']
        result += f"**Trecho de {source} - página {page}:** \n{page_content}\n\n"
    return result


# ------- AGENTS ------- #

library_agent = Agent(
    role="Especialista em Pesquisa em Biblioteca",
    goal="Encontre os trechos de livros que falem explicitamente do tópico fornecido.",
    backstory=(
        "Você atualmente é encarregado de usar a library_tool "
        "para encontrar trechos que falem do tópico fornecido. "
        "Seu objetivo é garantir que os trechos que você fornece "
        "tratem diretamente do tópico e não uma relação distante. "
    ),
    tools=[library_tool],
    allow_delegation=False,
    verbose=VERBOSE,
    llm=llm
)

revisor_agent = Agent(
    role="Revisor de Texto",
    goal="Revisar e melhorar a qualidade dos trechos fornecidos pelos Especialistas.",
    backstory=(
        "Você atualmente é encarregado de revisar e melhorar a qualidade "
        "dos trechos fornecidos pelos Especialistas em Pesquisa da Biblioteca. "
        "Seu objetivo é garantir que os trechos fornecidos são relevantes, "
        "precisos e falem do tópico fornecido pelo cliente."
    ),
    allow_delegation=False,
    verbose=VERBOSE,
    llm=llm,
)

# ------- TASKS ------- #

library_search_task = Task(
    description=(
        "Sua tarefa é usar a library_tool para encontrar trechos de livros que falem explicitamente "
        " do tópico: {text} "
    ),
    expected_output=(
        "Trechos de livros que falem explicitamente do tópico fornecido. "
        "Cada trecho deve ser claramente citado e incluída sua fonte, com o título do livro "
        "autor e número da página."
    ),
    agent=library_agent,
)

library_select_task = Task(
    description=(
        "Selecione os trechos que falam sobre o tópico: {text} "
        "Explique a relação ou indique caso não haja relação. Responda em português."
    ),
    expected_output=(
        "Tópico fornecido pelo cliente."
        "Trechos que falam do tópico fornecido pelo cliente, citando com sua fonte, incluindo o título do livro "
        "autor e número da página ou indicação caso não haja relação. "
        "Explicação onde o trecho fala diretamente sobre o tópico fornecido pelo cliente."
    ),
    agent=revisor_agent,
)

In [4]:
crew = Crew(
  agents=[library_agent, revisor_agent],
  tasks=[library_search_task, library_select_task],
  verbose=VERBOSE,
  memory=False,
)

In [5]:
input_text  = """O que falta aos pregadores hoje"""
input_text

'O que falta aos pregadores hoje'

In [6]:
inputs = {
    "text": input_text
}
result = crew.kickoff(inputs=inputs)

 

**Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill - página 77:** 
na sua fidelidade ao Pai  (prepa rou-se durante tanto tempo pa ra 
pregar por tão curto período); grande em sua submissão  ao Espírito 
(anda va ou parava de acordo com as orientações dele); grande nas 
afirm ações que fez  sobre o Filho (apon tando Jesus como “o Cordeiro 
de Deus, que tira o pecado do mundo ”, apesar de não te r-se avistado 
com ele antes).  
João era uma “Voz”. A maioria dos pregadores não passa de ecos, 
pois, se prestarmos bem atenção, saberemos dizer quais os livros 
que andaram lendo, e notaremos que citaram muito pouco do Livro. E 
hoje, só uma Voz, voz de um pr ofeta enviado do céu par a pregar aos 
pregadores , conseguiria despertar o co ração dos homens. S ó quem

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 99:** 
Monk Kidd, "confiando que a migalha da graça cairá".102 Reconhecendo humildemente que são 
proletários impotentes para alcançarem o desejo  do coraç

In [7]:
from IPython.display import display, Markdown

def get_task_output(task):
    return f"""
**Tarefa:** \n
{task.output.description} \n \n
**Saída:** \n
{task.output.raw_output}
"""

Markdown(get_task_output(library_search_task))



**Tarefa:** 

Sua tarefa é usar a library_tool para encontrar trechos de livros que falem explicitamente  do tópico: O que falta aos pregadores hoje  
 

**Saída:** 

**Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill - página 77:** 
na sua fidelidade ao Pai  (prepa rou-se durante tanto tempo pa ra 
pregar por tão curto período); grande em sua submissão  ao Espírito 
(anda va ou parava de acordo com as orientações dele); grande nas 
afirm ações que fez  sobre o Filho (apon tando Jesus como “o Cordeiro 
de Deus, que tira o pecado do mundo ”, apesar de não te r-se avistado 
com ele antes).  
João era uma “Voz”. A maioria dos pregadores não passa de ecos, 
pois, se prestarmos bem atenção, saberemos dizer quais os livros 
que andaram lendo, e notaremos que citaram muito pouco do Livro. E 
hoje, só uma Voz, voz de um pr ofeta enviado do céu par a pregar aos 
pregadores , conseguiria despertar o co ração dos homens. S ó quem

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 99:** 
Monk Kidd, "confiando que a migalha da graça cairá".102 Reconhecendo humildemente que são 
proletários impotentes para alcançarem o desejo  do coração sem ajuda di vina, eles são gratos 
pela menor migalha que cai da boca do pregador. 
Longas orações e palavras complicadas não convêm a maltrapilhos. Seu porta-voz é o 
cobrador de impostos no templo: "O Deus, sê propício a mim, pecador!" (Lc 18:13). O maltrapilho sabe que é o cobrad or de impostos e que recusar- se a admiti-lo faria dele um 
fariseu. Cobradores de impostos, zés-ninguém, vagabundos, mendigos de Jesus — os maltrapilhos sabem rir da sua vaidade de dese jarem ser percebidos e da vaidade de serem 
insignificantes.

**Trecho de Somos Todos Teólogos - RC Sproul - página 210:** 
Quando, porém, vier o Consolador, que eu vos enviarei da parte do Pai, o Espírito
da verdade, que dele procede, esse dará testemunho de mim; e vós também
testemunhareis, porque estais comigo desde o princípio.
Tenho-vos dito estas coisas para que não vos escandalizeis. Eles vos expulsarão
das sinagogas; mas vem a hora em que todo o que vos matar julgará com isso
tributar culto a Deus. Isto farão porque não conhecem o Pai, nem a mim. Ora, estas
coisas vos tenho dito para que, quando a hora chegar, vos recordeis de que eu vo-
las disse. Não vo-las disse desde o princípio, porque eu estava convosco (Jo 15.24-
16.4).
 
OUTRO CONSOLADOR

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 81:** 
vida e de amor que estão dentro de nós. E recai sobre nós o julgamento do Grande Senhor 
Executor de Mikado  sobre a garota que procrastinava perpe tuamente: "Jamais se sentirá falta 
dela; não, não: jamais se sentirá falta dela!" 
Cada um de nós paga um alto preço por nosso medo de cair de cara no chão. Ele assegura 
um estreitamento progressivo das nossas pe rsonalidades e impede a exploração e a 
experimentação. A medida que ficamos velhos seguimos fazendo apenas aquilo que fazemos 
bem. Não há crescimento em Jesus Cristo sem alguma dificuldade e embaraço. Se devemos nos 
manter crescendo, devemos permanecer correndo o risco de fracasso ao longo de toda a nossa


In [8]:
from IPython.display import display, Markdown

def get_task_output(task):
    return f"""
**Tarefa:** \n
{task.output.description} \n \n
**Saída:** \n
{task.output.raw_output}
"""

Markdown(get_task_output(library_select_task))



**Tarefa:** 

Selecione os trechos que falam sobre o tópico: O que falta aos pregadores hoje Explique a relação ou indique caso não haja relação. Responda em português. 
 

**Saída:** 

**Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill - página 77:** 
O trecho fala sobre a importância de ser uma "Voz" como João, um profeta enviado do céu para pregar aos pregadores e despertar o coração dos homens. Destaca a necessidade de pregadores serem mais do que simples ecos, citando muito pouco do Livro, e sim serem fiéis ao Pai, submissos ao Espírito e afirmarem sobre o Filho. Isso mostra a falta de pregadores hoje em serem verdadeiras vozes que apontam para Jesus.

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 99:** 
Neste trecho, é abordada a humildade dos maltrapilhos em reconhecerem sua impotência e dependerem da graça divina. Eles são gratos até pela menor migalha que cai da boca do pregador, demonstrando a falta de humildade e dependência de muitos pregadores hoje em dia.

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 81:** 
O texto destaca a importância de enfrentar o medo de fracassar e cair de cara no chão, pois é necessário correr riscos para crescer em Jesus Cristo. Isso ressalta a falta de coragem e disposição para enfrentar desafios que muitos pregadores enfrentam atualmente.

Esses trechos abordam a falta de pregadores hoje em serem verdadeiras vozes que apontam para Jesus, a falta de humildade e dependência da graça divina, e a falta de coragem para enfrentar desafios e crescer em Jesus Cristo.


In [9]:
Markdown(result)

**Trecho de Por que tarda o pleno Avivamento - Leonard Ravenhill - página 77:** 
O trecho fala sobre a importância de ser uma "Voz" como João, um profeta enviado do céu para pregar aos pregadores e despertar o coração dos homens. Destaca a necessidade de pregadores serem mais do que simples ecos, citando muito pouco do Livro, e sim serem fiéis ao Pai, submissos ao Espírito e afirmarem sobre o Filho. Isso mostra a falta de pregadores hoje em serem verdadeiras vozes que apontam para Jesus.

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 99:** 
Neste trecho, é abordada a humildade dos maltrapilhos em reconhecerem sua impotência e dependerem da graça divina. Eles são gratos até pela menor migalha que cai da boca do pregador, demonstrando a falta de humildade e dependência de muitos pregadores hoje em dia.

**Trecho de Evangelho Maltrapilho - Brennan Manning - página 81:** 
O texto destaca a importância de enfrentar o medo de fracassar e cair de cara no chão, pois é necessário correr riscos para crescer em Jesus Cristo. Isso ressalta a falta de coragem e disposição para enfrentar desafios que muitos pregadores enfrentam atualmente.

Esses trechos abordam a falta de pregadores hoje em serem verdadeiras vozes que apontam para Jesus, a falta de humildade e dependência da graça divina, e a falta de coragem para enfrentar desafios e crescer em Jesus Cristo.

In [10]:
print(crew.usage_metrics)

{'total_tokens': 26820, 'prompt_tokens': 18486, 'completion_tokens': 8334, 'successful_requests': 18}


In [11]:
#todo: criar um script python para rodar e retornar tudo isso depois desenvolver o contexto e relações bíblicas contexto histórico etc